**Loading the Model**

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 15.9 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
from torch.utils.data import DataLoader, Dataset

In [ ]:
#Loading the Domain Specific Trained Model 
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.load_state_dict(torch.load('Path to stored weights - Domain Training'))
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

**Dataset Preparation**

In [4]:
#Loading the datasets
!unzip '/content/drive/MyDrive/591 Project /cosmetics.csv.zip'
!unzip '/content/drive/MyDrive/591 Project /sephora_website_dataset.csv.zip'

Archive:  /content/drive/MyDrive/591 Project /cosmetics.csv.zip
  inflating: cosmetics.csv           
Archive:  /content/drive/MyDrive/591 Project /sephora_website_dataset.csv.zip
  inflating: sephora_website_dataset.csv  


In [5]:
import pandas as pd
df1 = pd.read_csv('/content/cosmetics.csv')
df2 = pd.read_csv('/content/sephora_website_dataset.csv')

In [6]:
df1.head()

,Label,Brand,Name,Price,Rank,Ingredients,Combination,Dry,Normal,Oily,Sensitive
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",1,1,1,1,1
1,Moisturizer,SK-II,Facial Treatment Essence,179,4.1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",1,1,1,1,1
2,Moisturizer,DRUNK ELEPHANT,Protini™ Polypeptide Cream,68,4.4,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",1,1,1,1,0
3,Moisturizer,LA MER,The Moisturizing Soft Cream,175,3.8,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",1,1,1,1,1
4,Moisturizer,IT COSMETICS,Your Skin But Better™ CC+™ Cream with SPF 50+,38,4.1,"Water, Snail Secretion Filtrate, Phenyl Trimet...",1,1,1,1,1


In [7]:
df2.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
0,2218774,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,5 x 0.16oz/5mL,4.0,4,3002,66.0,75.0,...,True,online only,no options,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...,1,0,0,0
1,2044816,Acqua Di Parma,Cologne,Colonia,0.7 oz/ 20 mL,4.5,76,2700,66.0,66.0,...,True,online only,- 0.7 oz/ 20 mL Spray - 1.7 oz/ 50 mL Eau d...,An elegant timeless scent filled with a fresh-...,no instructions,unknown,1,0,0,0
2,1417567,Acqua Di Parma,Perfume,Arancia di Capri,5 oz/ 148 mL,4.5,26,2600,180.0,180.0,...,True,online only,- 1oz/30mL Eau de Toilette - 2.5 oz/ 74 mL E...,Fragrance Family: Fresh Scent Type: Fresh Citr...,no instructions,Alcohol Denat.- Water- Fragrance- Limonene- Li...,1,0,0,0
3,1417617,Acqua Di Parma,Perfume,Mirto di Panarea,2.5 oz/ 74 mL,4.5,23,2900,120.0,120.0,...,True,online only,- 1 oz/ 30 mL Eau de Toilette Spray - 2.5 oz/...,Panarea near Sicily is an an island suspended ...,no instructions,unknown,1,0,0,0
4,2218766,Acqua Di Parma,Fragrance,Colonia Miniature Set,5 x 0.16oz/5mL,3.5,2,943,72.0,80.0,...,True,online only,no options,The Colonia Miniature Set comes in an iconic A...,Suggested Usage:-Fragrance is intensified by t...,Colonia: Alcohol Denat.- Water- Fragrance- Lim...,1,0,0,0


In [8]:
df2.rename(columns = {'ingredients':'Ingredients', 'category':'Label'}, inplace = True)

In [9]:
df2 = df2[['Ingredients', 'Label']]
df1 = df1[['Ingredients', 'Label']]

In [10]:
dataset = pd.concat([df1, df2])

In [11]:
dataset.head()

,Ingredients,Label
0,"Algae (Seaweed) Extract, Mineral Oil, Petrolat...",Moisturizer
1,"Galactomyces Ferment Filtrate (Pitera), Butyle...",Moisturizer
2,"Water, Dicaprylyl Carbonate, Glycerin, Ceteary...",Moisturizer
3,"Algae (Seaweed) Extract, Cyclopentasiloxane, P...",Moisturizer
4,"Water, Snail Secretion Filtrate, Phenyl Trimet...",Moisturizer


In [12]:
len(dataset)

10640

**Dataset Split**

**Model Training**

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= 'add train dataset',
    eval_dataset= 'add test dataset',
    tokenizer= tokenizer)

trainer.train()